In [1]:
import numpy as np

In [17]:
a = np.random.uniform(0, 1, 10).reshape(-1, 5)
a

array([[0.76003684, 0.01724832, 0.74101756, 0.67306081, 0.67825401],
       [0.12457276, 0.69597041, 0.81102444, 0.59237395, 0.98606556]])

In [9]:
a.shape

(2, 5)